# Facial emotion recognition

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
import cv2
from deepface import DeepFace

In [ ]:
face_model = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [ ]:
frame_list = []

In [ ]:
import cv2
from deepface import DeepFace

capture = cv2.VideoCapture("anyvideo")

#the loop depends on the frame rate
for i in range(1800):
    ret, frame = capture.read()
    if not ret:
        break  # If no frame is read, break the loop

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_model.detectMultiScale(gray_frame, 1.1, 5)

    for (x, y, width, height) in faces:
        face_roi = frame[y:y+height, x:x+width]  # Extract the face region of interest

        try:
            # Analyzing the face within the region of interest
            emotion = DeepFace.analyze(face_roi, actions=["emotion"], enforce_detection=False)
            dominant_emotion = emotion[0]["dominant_emotion"]
            cv2.putText(frame, dominant_emotion, (x, y + height + 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 225), 2)
            cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 255, 0), 2)
        except Exception as e:
            print("Error processing face:", str(e))

    frame_list.append(frame)  # Append the processed frame to the list

    if i == 0:  # Get video size from the first frame
        height, width, layers = frame.shape
        size = (width, height)

# Release the capture after processing
capture.release()

In [ ]:
output_path = "Emotions.avi"
output = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"DIVX"), 60, size)

In [ ]:
for frame in range(len(frame_list)):
  output.write(frame_list[frame])

In [ ]:
output.release()

# Speech to text

In [ ]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
apikey = "Zb5kuD44u2ssacFvZCBQn5pSK5hH5cYzaSlIz3wKLcFt"
url = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/9db59064-2906-44af-9e56-da50a4c6bfab"

In [ ]:
authenticator = IAMAuthenticator(apikey)

In [ ]:
stt = SpeechToTextV1(authenticator = authenticator)

In [ ]:
stt.set_service_url(url)

In [ ]:
import subprocess 
import os

#here we split the audio files into small groups
command = 'ffmpeg -i watsonx.mp4 -vn -ar 44100 -ac 2 -b:a 192k audio.mp3'
subprocess.call(command, shell=True)
command = 'ffmpeg -i audio.mp3 -f segment -segment_time 360 -c copy %03d.mp3'
subprocess.call(command, shell=True)

In [ ]:
#here we go through the audio files
files = []
for filename in os.listdir('.'):
    if filename.endswith(".mp3") and filename !='audio.mp3':
        files.append(filename)
files.sort()

In [ ]:
files

In [ ]:
results = []
for filename in files:
    with open(filename, 'rb') as f:
        res = stt.recognize(audio=f, content_type='audio/mp3', model='en-US_NarrowbandModel', inactivity_timeout=360).get_result()
        results.append(res)

In [ ]:
results

In [ ]:
textt = []
for file in results:
    for result in file['results']:
        textt.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [ ]:
textt

In [ ]:
#to save a text in a file
with open('output.txt', 'w') as out:
    out.writelines(textt)

# NLU

In [ ]:
#to read the text from the file
file_path = 'output.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    textt = file.read()

In [ ]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

In [ ]:
authenticator = IAMAuthenticator('Acws9345wBrFv-Adehqt9MOXYFiG9SHRx76Kh_d457r4')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator)

In [ ]:
natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/94c4168e-20b7-4605-84dc-658a93875153')

In [ ]:
response = natural_language_understanding.analyze(
    text=textt,
    features=Features(
        entities=EntitiesOptions(emotion=True, sentiment=True),
        keywords=KeywordsOptions(emotion=True, sentiment=True))).get_result()

In [ ]:
print(json.dumps(response, indent=2))